In [79]:
import pandas as pd
import dedupe
import os
import json

In [80]:
df_reference = pd.read_excel("../final/filtered_data_505(1).xlsx")
df_target = pd.read_excel("../final/chinese_enemy_data.xlsx")

In [81]:
# Fill missing data with empty strings
df_reference = df_reference.fillna("")
df_target = df_target.fillna("")

In [82]:
#o dict-of-dicts format
data_reference = df_reference.to_dict(orient='index')
data_target = df_target.to_dict(orient='index')

In [83]:
fields = [
    {'field': 'Product name', 'type': 'String', 'has_missing': True, 'weight': 2},
    {'field': 'Function', 'type': 'String', 'has_missing': True, 'weight': 1.5},
    {'field': 'Actuator size', 'type': 'String', 'has_missing': True, 'weight': 1},
    {'field': 'Coil connection', 'type': 'String', 'has_missing': True, 'weight': 1},
    {'field': 'Kv value [m³/h]', 'type': 'Price', 'has_missing': True, 'weight': 2},
    {'field': 'Orifice size [mm]', 'type': 'Price', 'has_missing': True, 'weight': 2},
    {'field': 'Max. Working Pressure [bar]', 'type': 'Price', 'has_missing': True, 'weight': 1},
    {'field': 'Supply voltage [V] AC', 'type': 'String', 'has_missing': True, 'weight': 1},
    {'field': 'Refrigerants', 'type': 'Categorical', 'has_missing': True, 'weight': 1},
]

In [84]:
# === Initialize Dedupe ===
deduper = dedupe.RecordLink(fields)

# Train or load settings
settings_file = 'valve_dedupe_settings.json'
training_file = 'valve_training.json'

if os.path.exists(settings_file):
    with open(settings_file, 'rb') as f:
        deduper.prepare_training(data_reference, data_target, training_file=training_file)
        deduper.read_settings(f)
else:
    deduper.prepare_training(data_reference, data_target, training_file=training_file)
    print("🔹 Starting interactive labeling (press Enter for unsure)...")
    dedupe.consoleLabel(deduper)
    deduper.train()
    with open(settings_file, 'wb') as f:
        deduper.write_settings(f)

# === Match target to reference ===
linked_records = deduper.match(data_reference, data_target, threshold=0.5)

# === Display top match for each target ===
for (ref_id, target_id), score in linked_records:
    ref_valve = df_reference.loc[ref_id]
    target_valve = df_target.loc[target_id]
    print(f"\n🔍 Target Valve: {target_valve['<ID>']} — {target_valve['Product name']}")
    print(f"✅ Closest Match: {ref_valve['<ID>']} — {ref_valve['Product name']}")
    print(f"Similarity Score: {round(score, 3)}")

ValueError: It looks like you are trying to use a variable definition composed of dictionaries. dedupe 3.0 uses variable objects directly. So instead of [{"field": "name", "type": "String"}] we now do [dedupe.variables.String("name")].

In [70]:
# Example    DataFrame
df = pd.read_excel("../final/filtered_data_505(1).xlsx")

# Dedupe requires a dict of dicts like:
# { record_id: {'field1': value, 'field2': value, ...}, ... }

# only valves
df = df.iloc[102:]

data_d = df.to_dict(orient='index')


In [71]:
columns = df.columns.tolist()
print(columns)

['<ID>', 'Product name', 'Actuator size', 'Actuator size [in]', 'Actuator system [mm]', 'Coil', 'Coil connection', 'Connection size [in]', 'Connection size [mm]', 'Connection type', 'Cv value [gal/min]', 'Cv value invert flow [gal/min]', 'Direction', 'Electrical connection', 'Enclosure rating IP', 'Frequency [Hz]', 'Function', 'Kv value [m³/h]', 'Kv value invert flow [m³/h]', 'Manual operation option', 'Max. Working Pressure [bar]', 'Max. Working Pressure [psig]', 'Media recommendations', 'Orifice size [mm]', 'Orifice size fraction [in]', 'Parts Program name', 'Power consumption [W]', 'Power consumption [W] 50Hz', 'Power consumption [W] 60Hz', 'Power consumption [W] DC', 'Refrigerants', 'Supply voltage [V] AC', 'Supply voltage [V] AC [max]', 'Supply voltage [V] AC [min]', 'Supply voltage [V] DC']


In [72]:
import dedupe

In [73]:
df = df.fillna('')             # replace NaNs with empty string
df = df.astype(str)            # convert ALL columns to string


In [74]:
fields = [
    dedupe.variables.String("Product name"),
    dedupe.variables.String("Actuator size"),
    dedupe.variables.String("Actuator size [in]"),
    dedupe.variables.String("Actuator system [mm]"),
    dedupe.variables.String("Coil"),
    dedupe.variables.String("Coil connection"),
    dedupe.variables.String("Connection size [in]"),
    dedupe.variables.String("Connection size [mm]"),
    dedupe.variables.String("Connection type"),
    dedupe.variables.String("Cv value [gal/min]"),
    dedupe.variables.String("Cv value invert flow [gal/min]"),
    dedupe.variables.String("Direction"),
    dedupe.variables.String("Electrical connection"),
    dedupe.variables.String("Enclosure rating IP"),
    dedupe.variables.String("Frequency [Hz]"),
    dedupe.variables.String("Function"),
    dedupe.variables.String("Kv value [m³/h]"),
    dedupe.variables.String("Kv value invert flow [m³/h]"),
    dedupe.variables.String("Manual operation option"),
    dedupe.variables.String("Max. Working Pressure [bar]"),
    dedupe.variables.String("Max. Working Pressure [psig]"),
    dedupe.variables.String("Media recommendations"),
    dedupe.variables.String("Orifice size [mm]"),
    dedupe.variables.String("Orifice size fraction [in]"),
    dedupe.variables.String("Parts Program name"),
    dedupe.variables.String("Power consumption [W]"),
    dedupe.variables.String("Power consumption [W] 50Hz"),
    dedupe.variables.String("Power consumption [W] 60Hz"),
    dedupe.variables.String("Power consumption [W] DC"),
    dedupe.variables.String("Refrigerants"),
    dedupe.variables.String("Supply voltage [V] AC"),
    dedupe.variables.String("Supply voltage [V] AC [max]"),
    dedupe.variables.String("Supply voltage [V] AC [min]"),
    dedupe.variables.String("Supply voltage [V] DC"),
]


In [75]:
deduper = dedupe.Dedupe(fields)

# Prepare training
deduper.prepare_training(data_d)

# Minimal auto training — we can skip manual labeling
deduper.train()


AttributeError: 'float' object has no attribute 'translate'